# Data exploration & preprocession

In [84]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
data=pd.read_csv(r"C:\Users\Microsoft\Downloads\IMDB Dataset.csv")
data

review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]

In [85]:
data.columns 

Index(['review', 'sentiment'], dtype='object')

In [87]:
data = data.drop(columns=data.columns[3:38])

In [88]:
data

review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [90]:
data.describe()

review sentiment
count                                               50000     50000
unique                                              49582         2
top     Loved today's show!!! It was a variety and not...  positive
freq                                                    5     25000

In [92]:
print(data['sentiment'].value_counts())

sentiment
positive    25000
negative    25000
Name: count, dtype: int64


# preprocess

In [93]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from nltk.tokenize import word_tokenize

In [94]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Microsoft\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Microsoft\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [96]:
def preprocess_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    
    # Load stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    return ' '.join(filtered_tokens)

data['review'] = data['review'].astype(str)
data['cleaned_text'] = data['review'].apply(preprocess_text)
data['cleaned_text']

0        one reviewers mentioned watching oz episode ho...
1        wonderful little production br br filming tech...
2        thought wonderful way spend time hot summer we...
3        basically family little boy jake thinks zombie...
4        petter mattei love time money visually stunnin...
                               ...                        
49995    thought movie right good job creative original...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movies high art fans exp...
Name: cleaned_text, Length: 50000, dtype: object

In [97]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
x= tfidf_vectorizer.fit_transform(data['cleaned_text'])

# The labels for classification
y = data['sentiment']

#  naive Bayes Model for Text Classification

In [98]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train,y_train

(<40000x99250 sparse matrix of type '<class 'numpy.float64'>'
 	with 3886332 stored elements in Compressed Sparse Row format>,
 39087    negative
 30893    negative
 45278    positive
 16398    negative
 13653    negative
            ...   
 11284    positive
 44732    positive
 38158    negative
 860      positive
 15795    positive
 Name: sentiment, Length: 40000, dtype: object)

In [99]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

In [100]:
# Fit the MNb classifier to the training data
nb = MultinomialNB()
nb.fit(x_train, y_train)

MultinomialNB()

In [102]:
if y_train.isnull().any():
    print("NaN values found in y_train!")
    # Remove rows with NaN values
    y_train = y_train.dropna()




In [103]:
y_train

39087    negative
30893    negative
45278    positive
16398    negative
13653    negative
           ...   
11284    positive
44732    positive
38158    negative
860      positive
15795    positive
Name: sentiment, Length: 40000, dtype: object

In [104]:
!pip install textblob


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [105]:
from textblob import TextBlob

In [106]:
# Function to get sentiment
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

# Apply sentiment analysis
data['sentiment'] = data['cleaned_text'].apply(get_sentiment)

# Display sentiment distribution
print(data['sentiment'].value_counts())


sentiment
positive    36652
negative    13305
neutral        43
Name: count, dtype: int64


# evaluation

In [107]:

# Calculate evaluation metrics
from sklearn.metrics import classification_report

# Make predictions on test data
y_pred = naive_bayes.predict(x_test)

print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

    negative       0.86      0.88      0.87      4961
    positive       0.88      0.85      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



 Naive Bayes classifier can accurately categorize blog posts with proper preprocessing and feature extraction.
- Performance metrics like accuracy, precision, recall, and F1-score evaluate the model's effectiveness.
- Class imbalance and text preprocessing challenges need careful handling.
- Sentiment analysis reveals the emotional tone of blog posts, valuable for content strategy and reader engagement.
- Combining text classification and sentiment analysis provides powerful insights from textual data.
